In [21]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
text = open(r'C:\Users\asus\Desktop\train.txt','r').read()
text=text.split('. . O')

In [22]:
data=[]
unwanted_tags=['#',
 '$',
 "''",
 '(',
 ')',
 ',',
 '.',
 ':']
for i in text:
    train_x=[]
    T=list(i.split('\n'))
    for j in T:
        
        m=j.split()
        if len(m)<=1 :
            continue
        elif m[1] in unwanted_tags:
            continue
        train_x.append((m[0],m[1]))
    data.append(train_x)
        
            
    

In [23]:
# split data into training and validation set in the ratio 95:5
random.seed(1234)
train_set, test_set = train_test_split(data, train_size=0.95, test_size=0.05)

print("Training Set Length -", len(train_set))
print("Testing Set Length -", len(test_set))
print("-" * 100)
print("Training Data -\n")
print(train_set[:10])

Training Set Length - 8276
Testing Set Length - 436
----------------------------------------------------------------------------------------------------
Training Data -

[[('Since', 'IN'), ('last', 'JJ'), ('week', 'NN'), ("'s", 'POS'), ('junk-bond', 'NN'), ('market', 'NN'), ('debacle', 'NN'), ('many', 'JJ'), ('new', 'JJ'), ('issues', 'NNS'), ('of', 'IN'), ('high-yield', 'JJ'), ('high-risk', 'JJ'), ('corporate', 'JJ'), ('bonds', 'NNS'), ('have', 'VBP'), ('either', 'DT'), ('been', 'VBN'), ('scaled', 'VBN'), ('back', 'RB'), ('delayed', 'VBN'), ('or', 'CC'), ('dropped', 'VBD')], [('The', 'DT'), ('following', 'VBG'), ('were', 'VBD'), ('among', 'IN'), ('yesterday', 'NN'), ("'s", 'POS'), ('offerings', 'NNS'), ('and', 'CC'), ('pricings', 'NNS'), ('in', 'IN'), ('the', 'DT'), ('U.S.', 'NNP'), ('and', 'CC'), ('non-U.S.', 'JJ'), ('capital', 'NN'), ('markets', 'NNS'), ('with', 'IN'), ('terms', 'NNS'), ('and', 'CC'), ('syndicate', 'NN'), ('manager', 'NN'), ('as', 'IN'), ('compiled', 'VBN'), ('by', '

In [24]:
# Getting list of train and test tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
print("Train Tagged Words - ", len(train_tagged_words))

test_tagged_words = [tup[0] for sent in test_set for tup in sent]
print("test Tagged Words - ", len(test_tagged_words))

Train Tagged Words -  178135
test Tagged Words -  9114


In [25]:
# Let's have a look at the tagged words in the training set
train_tagged_words[:10]

[('Since', 'IN'),
 ('last', 'JJ'),
 ('week', 'NN'),
 ("'s", 'POS'),
 ('junk-bond', 'NN'),
 ('market', 'NN'),
 ('debacle', 'NN'),
 ('many', 'JJ'),
 ('new', 'JJ'),
 ('issues', 'NNS')]

In [26]:
# tokens in the train set - train_tagged_words
train_tagged_tokens = [tag[0] for tag in train_tagged_words]
train_tagged_tokens[:10]

['Since',
 'last',
 'week',
 "'s",
 'junk-bond',
 'market',
 'debacle',
 'many',
 'new',
 'issues']

In [27]:
# POS tags for the tokens in the train set - train_tagged_words

train_tagged_pos_tokens = [tag[1] for tag in train_tagged_words]
set(sorted(train_tagged_pos_tokens))

{'CC',
 'CD',
 'DT',
 'EX',
 'FW',
 'IN',
 'JJ',
 'JJR',
 'JJS',
 'MD',
 'NN',
 'NNP',
 'NNPS',
 'NNS',
 'PDT',
 'POS',
 'PRP',
 'PRP$',
 'RB',
 'RBR',
 'RBS',
 'RP',
 'SYM',
 'TO',
 'UH',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBP',
 'VBZ',
 'WDT',
 'WP',
 'WP$',
 'WRB',
 '``'}

In [28]:
# building the train vocabulary to a set
training_vocabulary_set = set(train_tagged_tokens)

In [29]:
# building the POS tags to a set
training_pos_tag_set = set(train_tagged_pos_tokens)

In [30]:
#tags are present in training data
print(len(training_pos_tag_set))

36


In [31]:
# let's check how many words are present in vocabulary
print(len(training_vocabulary_set))

18580


In [32]:
# compute emission probability for a given word for a given tag
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1] == tag]
    tag_count = len(tag_list)    
    word_given_tag_list = [pair[0] for pair in tag_list if pair[0] == word]    
    word_given_tag_count = len(word_given_tag_list)    
    
    return (word_given_tag_count, tag_count)

In [33]:
# compute transition probabilities of a previous and next tag
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    
    t1_tags_list = [tag for tag in tags if tag == t1]
    t1_tags_count = len(t1_tags_list)
    
    t2_given_t1_list = [tags[index+1] for index in range(len(tags)-1) if tags[index] == t1 and tags[index+1] == t2]
    t2_given_t1_count = len(t2_given_t1_list)
    
    return(t2_given_t1_count, t1_tags_count)

In [34]:
# computing P(w/t) and storing in [Tags x Vocabulary] matrix. This is a matrix with dimension
# of len(training_pos_tag_set) X en(training_vocabulary_set)

len_pos_tags = len(training_pos_tag_set)
len_vocab = len(training_vocabulary_set)

In [35]:
# creating t x t transition matrix of training_pos_tag_set
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len_pos_tags, len_pos_tags), dtype='float32')
for i, t1 in enumerate(list(training_pos_tag_set)):
    for j, t2 in enumerate(list(training_pos_tag_set)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [36]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(training_pos_tag_set), index=list(training_pos_tag_set))

In [37]:
# Let's have a glimpse into the transition matrix
tags_df

,VBN,NN,TO,NNPS,UH,RP,POS,RBR,VBP,MD,...,PDT,SYM,RB,FW,RBS,EX,JJS,WDT,IN,VBD
VBN,0.031098,0.093295,0.104543,0.000221,0.000000,0.000882,0.000000,0.001764,0.001323,0.001985,...,0.000441,0.000000,0.046096,0.000000,0.000221,0.000221,0.000441,0.000662,0.375827,0.003970
NN,0.017030,0.129122,0.043832,0.000523,0.000035,0.000105,0.021916,0.004153,0.004537,0.019717,...,0.000035,0.000105,0.030850,0.000174,0.000384,0.001047,0.000384,0.013261,0.278485,0.056325
TO,0.001036,0.020510,0.000207,0.000414,0.000000,0.000000,0.000000,0.000207,0.000000,0.000000,...,0.000000,0.000000,0.009944,0.000000,0.000414,0.000000,0.000000,0.000622,0.003936,0.000207
NNPS,0.012407,0.079404,0.027295,0.007444,0.000000,0.000000,0.034739,0.000000,0.081886,0.037221,...,0.000000,0.000000,0.014888,0.000000,0.000000,0.000000,0.002481,0.014888,0.116625,0.141439
UH,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.066667,0.000000,0.066667,0.000000,...,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000
RP,0.012987,0.064935,0.012987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.298701,0.000000
POS,0.012545,0.419355,0.000597,0.002389,0.000000,0.000000,0.000000,0.000597,0.000000,0.000597,...,0.000000,0.000000,0.008363,0.000597,0.003584,0.000000,0.022700,0.000000,0.002987,0.005974
RBR,0.029703,0.006601,0.006601,0.000000,0.000000,0.000000,0.003300,0.003300,0.003300,0.000000,...,0.000000,0.000000,0.128713,0.000000,0.000000,0.000000,0.003300,0.003300,0.122112,0.006601
VBP,0.151802,0.034583,0.054969,0.000364,0.000000,0.001820,0.000000,0.004368,0.001092,0.002184,...,0.001092,0.000000,0.168183,0.000000,0.001456,0.002184,0.000000,0.002912,0.102293,0.010193
MD,0.000487,0.000487,0.004380,0.000000,0.000000,0.000000,0.000000,0.000973,0.000000,0.000000,...,0.000000,0.000000,0.177129,0.000000,0.000000,0.000000,0.000000,0.000000,0.006813,0.002433


In [38]:
def Vanilla_Viterbi(words, train_bag = train_tagged_words):
    state = []
    
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [28]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1, len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [29]:
# tagging the test sentences
start = time.time()
tagged_seq = Vanilla_Viterbi(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
vanilla_viterbi_word_check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
vanilla_viterbi_accuracy = len(vanilla_viterbi_word_check)/len(tagged_seq) * 100
print('Vanilla Viterbi Algorithm Accuracy: ', vanilla_viterbi_accuracy)

Time taken in seconds:  182.55433869361877
Vanilla Viterbi Algorithm Accuracy:  96.3855421686747


In [30]:
# let's check the incorrectly tagged words
incorrect_tagged_words = [j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

print("Total Incorrect Tagged Words :", len(incorrect_tagged_words))
print("\n")
print("Incorrect Tagged Words :", incorrect_tagged_words)

Total Incorrect Tagged Words : 6


Incorrect Tagged Words : [(('executive', 'NN'), ('executive', 'JJ')), (('Pride', 'VBN'), ('Pride', 'NNP')), (('nonexecutive', 'VBN'), ('nonexecutive', 'JJ')), (('that', 'WDT'), ('that', 'IN')), (('refined', 'VBN'), ('refined', 'JJ')), (('paid', 'VBN'), ('paid', 'VBD'))]


In [31]:
# Unknown words 

test_vocabulary_set = set([t for t in test_tagged_words])

unknown_words = list(test_vocabulary_set - training_vocabulary_set)
print("Total Unknown words :", len(unknown_words))
print("\n")
print("Unknown Words :", unknown_words)

Total Unknown words : 3


Unknown Words : ['Pride', 'refined', 'nonexecutive']
